In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import requests

In [0]:

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Get Stock Data") \
    .getOrCreate()


In [0]:
# Make the request
api_string = "https://api.freeapi.app/api/v1/public/stocks?page=1&limit={pageLimit}&inc=Symbol%2CName&query={companyName}"
# company_name = ['tata','birla','infosys','wipro','reliance','adani','hdfc','icici','sbi','bharat']
company_name = ['tata']

In [0]:
#fetching associated companies
flag = 0
for company in company_name:
    response = requests.get(api_string.format(pageLimit=1,companyName=company))
    data_val = response.json()
    totalpages = data_val["data"]["totalPages"]
    response = requests.get(api_string.format(pageLimit=totalpages,companyName=company))
    data = response.json()
    if flag==0:
        stockmaster = spark.createDataFrame(data["data"]["data"])
        flag=1
    else:
        stockmaster2 = spark.createDataFrame(data["data"]["data"])
        stockmaster = stockmaster.union(stockmaster2)

In [0]:
# from pyspark.sql.types import StructType, StructField, StringType, FloatType

# schema = StructType([
#     StructField("Name", StringType(), True),
#     StructField("Symbol", StringType(), True),
#     StructField("BookValue", StringType(), True),
#     StructField("CurrentPrice", StringType(), True),
#     StructField("DividendYield", StringType(), True),
#     StructField("FaceValue", StringType(), True),
#     StructField("HighLow", StringType(), True),
#     StructField("ISIN", StringType(), True),
#     StructField("ListingDate", StringType(), True),  # Depending on format, might need DateType
#     StructField("MarketCap", StringType(), True),
#     StructField("ROCE", StringType(), True),
#     StructField("ROE", StringType(), True),
#     StructField("StockPE", StringType(), True)
# ])

# df_details = spark.createDataFrame([], schema)
# display(df_details)

Name,Symbol,BookValue,CurrentPrice,DividendYield,FaceValue,HighLow,ISIN,ListingDate,MarketCap,ROCE,ROE,StockPE


In [0]:
#fetching Stock details
STOCK_DETAILS_API = "https://api.freeapi.app/api/v1/public/stocks/{companySymbol}"

flag = 0

for rows in stockmaster.toLocalIterator():
    response = requests.get(STOCK_DETAILS_API.format(companySymbol=rows["Symbol"]))
    data = response.json()
    stockdetails = [data["data"]]
    if flag==0:
        stockdetailsdf = spark.createDataFrame(stockdetails)
        flag=1
    else:
        #df1.unionByName(df2, allowMissingColumns=True)
        stockdetailsdf2 = spark.createDataFrame(stockdetails)
        stockdetailsdf = stockdetailsdf.unionAll(stockdetailsdf2)

In [0]:
display(stockdetailsdf)

BookValue,CurrentPrice,DividendYield,FaceValue,HighLow,ISIN,ListingDate,MarketCap,Name,ROCE,ROE,StockPE,Symbol,High,Low,date
840,1113,1.54 %,10,"1,123 / 912",INE092A01019,01-Apr-99,null,Tata Chemicals Limited,11.6 %,12.0 %,14.4,TATACHEM,1123,912,2024-05-15
99,321,0.93 %,1,322 / 199,INE493A01027,16-Feb-00,null,Tata Coffee Limited,9.78 %,10.1 %,33.4,TATACOFFEE,322,199,2024-05-15
51,1781,1.18 %,10,"1,957 / 1,157",INE151A01013,12-Apr-95,null,Tata Communications Limited,23.9 %,142 %,38.5,TATACOMM,1957,1157,2024-05-15
174,1080,0.77 %,1,"1,081 / 685",INE192A01025,18-Nov-98,null,TATA CONSUMER PRODUCTS LIMITED,9.32 %,6.86 %,81.3,TATACONSUM,1081,685,2024-05-15
45,8740,0.69 %,10,"9,200 / 5,883",INE670A01012,17-Sep-03,null,Tata Elxsi Limited,60.0 %,39.3 %,533,TATAELXSI,9200,5883,2024-05-15
4691,4297,1.11 %,10,"4,737 / 1,730",INE672A01018,24-Jun-98,null,Tata Investment Corporation Limited,1.42 %,1.28 %,66.9,TATAINVEST,4737,1730,2024-05-15
59,1078,0.46 %,10,"1,085 / 721",INE056C01010,27-May-03,null,Tata Metaliks Limited,43.3 %,306 %,28.2,TATAMETALI,1085,721,2024-05-15
161,792,0.27 %,2,802 / 381,INE155A01022,22-Jul-98,null,Tata Motors Limited,5.95 %,5.62 %,18.1,TATAMOTORS,802,381,2024-05-15
1086,526,0.42 %,2,537 / 202,IN9155A01020,05-Nov-08,null,Tata Motors Limited,%,-3.68 %,,TATAMTRDVR,537,202,2024-05-15
94,333,0.61 %,1,347 / 182,INE245A01021,03-Apr-96,null,Tata Power Company Limited,11.7 %,12.6 %,31.2,TATAPOWER,347,182,2024-05-15


In [0]:
# Cleaning Data
from pyspark.sql.functions import regexp_replace

#Removing INR symbol
stockdetailsdf = stockdetailsdf.withColumn("CurrentPrice", regexp_replace("CurrentPrice", "₹ ", ""))
stockdetailsdf = stockdetailsdf.withColumn("BookValue", regexp_replace("BookValue", "₹ ", ""))
stockdetailsdf = stockdetailsdf.withColumn("BookValue", regexp_replace("BookValue", "₹", ""))
stockdetailsdf = stockdetailsdf.withColumn("FaceValue", regexp_replace("FaceValue", "₹ ", ""))
stockdetailsdf = stockdetailsdf.withColumn("HighLow", regexp_replace("HighLow", "₹ ", ""))
stockdetailsdf = stockdetailsdf.withColumn("MarketCap", regexp_replace("MarketCap", "₹ ", ""))

In [0]:

#Splitting High Lows
from pyspark.sql.functions import split

stockdetailsdf = stockdetailsdf.withColumn("High", split(stockdetailsdf["HighLow"], "/").getItem(0))
stockdetailsdf = stockdetailsdf.withColumn("Low", split(stockdetailsdf["HighLow"], "/").getItem(1))

In [0]:
#Removing Spaces and Commas
stockdetailsdf = stockdetailsdf.withColumn("CurrentPrice", regexp_replace("CurrentPrice", ",", ""))
stockdetailsdf = stockdetailsdf.withColumn("BookValue", regexp_replace("BookValue", ",", ""))
stockdetailsdf = stockdetailsdf.withColumn("FaceValue", regexp_replace("FaceValue", ",", ""))
stockdetailsdf = stockdetailsdf.withColumn("MarketCap", regexp_replace("MarketCap", ",", ""))
stockdetailsdf = stockdetailsdf.withColumn("High", regexp_replace("High", ",", ""))
stockdetailsdf = stockdetailsdf.withColumn("Low", regexp_replace("Low", ",", ""))
stockdetailsdf = stockdetailsdf.withColumn("High", regexp_replace("High", " ", ""))
stockdetailsdf = stockdetailsdf.withColumn("Low", regexp_replace("Low", " ", ""))

In [0]:
from pyspark.sql.functions import col
stockdetailsdf = stockdetailsdf.withColumn("CurrentPrice", col("CurrentPrice").cast("int"))
stockdetailsdf = stockdetailsdf.withColumn("BookValue", col("BookValue").cast("int"))
stockdetailsdf = stockdetailsdf.withColumn("FaceValue", col("FaceValue").cast("int"))
stockdetailsdf = stockdetailsdf.withColumn("MarketCap", col("MarketCap").cast("int"))
stockdetailsdf = stockdetailsdf.withColumn("High", col("High").cast("int"))
stockdetailsdf = stockdetailsdf.withColumn("Low", col("Low").cast("int"))

In [0]:
from pyspark.sql.functions import lit, current_date
stockdetailsdf = stockdetailsdf.withColumn("date", lit(current_date()))

In [0]:
import pyodbc 

cnxn = pyodbc.connect("Driver={/usr/local/lib/libmsodbcsql.18.dylib};Server=sakettestserver.database.windows.net;UID=saket;PWD=Mansa@0902;Database=demo;")